# Unit 4 Integrating MCP Servers to an OpenAI Agent

## Introduction & Lesson Overview

Welcome\! Now that you’ve built your MCP server and exposed your shopping list tools, it’s time to make them available to an OpenAI agent. In this lesson, you’ll learn how to connect your MCP server to an agent using both local (stdio) and remote (SSE) transports, how to provide the server to the agent, and how to test that the integration works as expected.

By the end of this lesson, you’ll be able to:

  * Connect an OpenAI agent to your MCP server using both `stdio` and `SSE` transports.
  * Provide the MCP server to the agent so it can discover and use your tools.
  * Run and test the integration, verifying that the agent can answer queries using your shopping list service.

Let’s walk through each step in detail.

## Connecting the Agents SDK to an MCP Server via Stdio

The simplest way to connect your MCP server to an agent is by using the `stdio` transport. This is ideal for local development, where your server runs as a subprocess on the same machine as your agent. Communication happens over standard input and output, making it fast and easy to set up.

Here’s how you can launch your MCP server and connect to it via `stdio` using the OpenAI Agents SDK:

```python
import asyncio
from agents.mcp import MCPServerStdio

async def main():
    # Define how to start your MCP server as a subprocess
    server_params = {
        "command": "python",
        "args": ["mcp_server.py"]
    }
    # Create a connection to the MCP server using stdio
    async with MCPServerStdio(params=server_params) as mcp_server:
        # Your agent code goes here...

if __name__ == "__main__":
    asyncio.run(main())
```

  * The `command` and `args` specify how to launch your MCP server script.
  * The `MCPServerStdio` context manager handles starting and stopping the server process for you.

This setup is perfect for development and testing on your own machine.

## Connecting the Agents SDK to an MCP Server via SSE

If your MCP server is running remotely—perhaps on another machine or in the cloud—you’ll want to use the `SSE` (Server-Sent Events) transport. This allows the agent to communicate with your server over HTTP, making it suitable for distributed or production environments.

Here’s how to connect using SSE:

```python
import asyncio
from agents.mcp import MCPServerSse

async def main():
    # Specify the URL where your MCP server is running
    server_params = {"url": "http://localhost:3000/sse"}
    # Create a connection to the MCP server using SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        # Your agent code goes here...

if __name__ == "__main__":
    asyncio.run(main())
```

  * Replace the URL with the address of your running MCP server.
  * The `MCPServerSse` context manager manages the HTTP connection for you.

This approach is great for connecting to servers that are not running on your local machine.

## Providing the MCP Server to the Agent

Once you have an `mcp_server` object—whether from `MCPServerStdio` or `MCPServerSse`—you can provide it to your agent. The agent will automatically discover all the tools your server exposes, read their documentation and input schemas, and use them to answer user queries.

Here’s a complete example using the `stdio` transport:

```python
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio

async def main():
    server_params = {
        "command": "python",
        "args": ["your_M.py"]
    }
    async with MCPServerStdio(params=server_params) as mcp_server:
        # Create the agent and provide the MCP server
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            model="gpt-4.1",
            mcp_servers=[mcp_server]  # List of MCP servers available
        )
        # Run the agent with a query
        result = await Runner.run(agent, "Give me my shopping list")
        
        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())
```

  * The `mcp_servers` argument is a list, so you can provide one or more MCP server connections.
  * The agent will aggregate all available tools from the connected servers.

When you run the script, the agent connects to your MCP server, discovers the tools, and uses them to answer the query.
You can use the same approach with `MCPServerSse` by swapping out the context manager.

## How the Agent Discovers and Uses Your Tools

When you provide the MCP server to the agent, the agent automatically connects and fetches all available tools. It reads the documentation and input schemas you defined with the `@mcp.tool()` decorator. This means the agent knows what each tool does and how to use it—no extra programming required.

For example:

  * If you ask, “Give me my shopping list”, the agent will recognize it can use the `fetch_items` tool.
  * If you say, “Add 3 bananas to my shopping list”, the agent will use the `add_item` tool with the correct parameters.

This automatic discovery and aggregation of tools is what makes MCP integration so powerful. The agent can flexibly use any tool you expose, based on the user’s request.

## Lesson Summary & Next Steps

In this lesson, you learned how to connect an OpenAI agent to your MCP server using both `stdio` and `SSE` transports. You saw how to provide the MCP server to the agent, allowing it to automatically discover and use your shopping list tools in response to natural language queries. You also learned how to run and test the integration, verifying that your tools are accessible to the agent.

You’re now ready to practice these skills by building and testing your own agent-server integrations. This is a major step forward—your tools are now available to intelligent agents that can use them in flexible, conversational ways. In the next exercises, you’ll get hands-on experience with these integrations and deepen your understanding even further.

## Connecting Your Agent to MCP Server

t’s finally time to put MCP and Agents together! Your first task is to modify the provided main.py so that our OpenAI agent connects to the shopping list MCP server using stdio. To do this, you will:

Import the correct class for stdio-based MCP server connections from the OpenAI Agents SDK.
Define the server parameters so the agent knows how to launch the MCP server as a subprocess.
Use an async context manager to handle the MCP server connection.
Make sure the agent is created with the connected MCP server in its mcp_servers list.
Once you’ve made these changes, the agent should be able to run a query like “Give me my shopping list” and get a response using the tools from your MCP server. Give it a try and see your integration in action!

```python
# main.py
import asyncio
from agents import Agent, Runner
# TODO: Import MCPServerStdio from agents.mcp


async def main():
    # TODO: Define the server parameters (command and args for mcp_server.py)

    # TODO: Use an async with statement to connect to the MCP server via stdio and assign the connection to a variable
    
    # TODO: Inside the context, create the agent and provide the MCP server
    agent = Agent(
        name="Shopping Assistant",
        instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
        model="gpt-4.1",
        # TODO: Add the mcp_servers parameter with the connected MCP server
    )

    # Run the agent with the single query
    result = await Runner.run(
        starting_agent=agent,
        input="Give me my shopping list"
    )

    # Print the final output
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

# mcp_server.py
```

All right, let's connect your OpenAI agent to the MCP server using the `stdio` transport\! This is a great way to test your integration locally.

Here's the completed `main.py` that accomplishes the task:

```python
# main.py
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerStdio # TODO: Import MCPServerStdio from agents.mcp


async def main():
    # TODO: Define the server parameters (command and args for mcp_server.py)
    server_params = {
        "command": "python",
        "args": ["mcp_server.py"] # This assumes mcp_server.py is in the same directory
    }

    # TODO: Use an async with statement to connect to the MCP server via stdio and assign the connection to a variable
    async with MCPServerStdio(params=server_params) as mcp_server:
    
        # TODO: Inside the context, create the agent and provide the MCP server
        agent = Agent(
            name="Shopping Assistant",
            instructions="You are an assistant that uses shopping list tools to help manage a shopping list.",
            model="gpt-4.1", # Ensure you have access to this model or change to one you have
            mcp_servers=[mcp_server] # TODO: Add the mcp_servers parameter with the connected MCP server
        )

        # Run the agent with the single query
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

**Explanation of Changes:**

1.  **`from agents.mcp import MCPServerStdio`**: This line imports the necessary class for connecting to an MCP server via standard I/O.
2.  **`server_params` dictionary**: This dictionary tells the `MCPServerStdio` how to execute your `mcp_server.py` script.
      * `"command": "python"`: Specifies that the command to run is `python`.
      * `"args": ["mcp_server.py"]`: Provides the arguments to the `python` command, which in this case is the name of your server script.
3.  **`async with MCPServerStdio(params=server_params) as mcp_server:`**: This is an asynchronous context manager. It will:
      * Automatically start your `mcp_server.py` as a subprocess when entering the `async with` block.
      * Manage the standard input/output streams for communication between the client and the server.
      * Assign the connected MCP server object to the `mcp_server` variable.
      * Automatically stop the `mcp_server.py` subprocess when exiting the `async with` block (e.g., when `main` function finishes).
4.  **`mcp_servers=[mcp_server]`**: Inside the `Agent` constructor, the `mcp_servers` parameter is a list where you pass the `mcp_server` object. This tells the agent about the available MCP server and its exposed tools.

**To run this, you'll also need your `mcp_server.py` file to be present in the same directory.** I'll provide the last version of `mcp_server.py` for completeness:

```python
# mcp_server.py
from mcp.server.fastmcp import FastMCP
from shopping_list import ShoppingListService
import uuid # Add this import if not already present in your actual mcp_server.py

# Create an MCP server
mcp = FastMCP(
    name="Shopping List",
    description="Provides tools to manage a shopping list: add items, remove items, mark purchased, and list items",
    log_level="ERROR" # Keeping this from previous exercise
)

# Create a shopping list service
shopping_list = ShoppingListService()


@mcp.tool()
def add_item(name: str, quantity: int) -> str:
    """
    Add a shopping list item.
    
    Parameters:
      name (str): The name of the item.
      quantity (int): The quantity to add.
      
    Returns:
      str: The unique ID of the newly added item.
    """
    return shopping_list.add_item(name, quantity)


@mcp.tool()
def remove_item(item_id: str) -> bool:
    """
    Remove an item from the shopping list.
    
    Parameters:
      item_id (str): The unique ID of the item.
      
    Returns:
      bool: True if the item was removed successfully; otherwise False.
    """
    return shopping_list.remove_item(item_id)


@mcp.tool()
def mark_purchased(item_id: str, purchased: bool = True) -> bool:
    """
    Mark an item as purchased or not.
    
    Parameters:
      item_id (str): The unique ID of the item.
      purchased (bool, optional): True to mark as purchased; False otherwise. Defaults to True.
      
    Returns:
      bool: True if the update succeeded; otherwise False.
    """
    return shopping_list.set_purchased(item_id, purchased)


@mcp.tool()
def fetch_items(purchased: bool = None) -> list:
    """
    Retrieve all shopping list items, optionally filtered by purchased status.
    
    Parameters:
        purchased (bool, optional): Filter items by purchased status.
            None returns all items,
            True returns only purchased items,
            False returns only unpurchased items.
    
    Returns:
        list: A list of shopping items matching the filter criteria.
    """
    return shopping_list.get_items(purchased)


if __name__ == "__main__":
    # IMPORTANT: For stdio connection, the server should run with transport="stdio"
    # The client (main.py) will launch this as a subprocess.
    mcp.run(transport="stdio")
    
```

You'll also need the `shopping_list.py` file as provided in previous exercises. When you run `main.py`, it will automatically start `mcp_server.py` and then interact with it.

## Switching to Remote SSE Integration

```python
# main.py

# mcp_server.py
```

## Multi Step Shopping List Challenge

```python
# main.py

# mcp_server.py
```